In [6]:
import pandas as pd
import requests
from citipy import citipy
import sys, os

sys.path.append(os.path.abspath(os.path.join('../../..')))
from apikeys import weather_api_key

In [7]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
coordinates = list(lat_lngs)

In [8]:
# Create a list of cities, from the nearest city to each coordinate
cities = []

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    if city not in cities:
        cities.append(city)

len(cities)

727

In [9]:
# Set up base API request URL
url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + weather_api_key

# Create an empty list to hold the weather data.
city_data = []

# Create counters.
record_count = 1
set_count = 1

In [10]:
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_desc = city_weather["weather"][0]["description"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                            "Country": city_country,
                            "Lat": city_lat,
                            "Lng": city_lng,
                            "Max Temp": city_max_temp,
                            "Humidity": city_humidity,
                            "Cloudiness": city_clouds,
                            "Wind Speed": city_wind,
                            "Current Description": city_desc})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | portland
Processing Record 2 of Set 1 | busselton
Processing Record 3 of Set 1 | havre-saint-pierre
Processing Record 4 of Set 1 | ayagoz
Processing Record 5 of Set 1 | mataura
Processing Record 6 of Set 1 | vardo
Processing Record 7 of Set 1 | aklavik
Processing Record 8 of Set 1 | rikitea
Processing Record 9 of Set 1 | hobart
Processing Record 10 of Set 1 | saint paul
Processing Record 11 of Set 1 | koroni
Processing Record 12 of Set 1 | kaeo
Processing Record 13 of Set 1 | longyearbyen
Processing Record 14 of Set 1 | albany
Processing Record 15 of Set 1 | dikson
Processing Record 16 of Set 1 | mount gambier
Processing Record 17 of Set 1 | barrow
Processing Record 18 of Set 1 | puerto ayora
Processing Record 19 of Set 1 | port elizabeth
Processing Record 20 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 21 of Set 1 | ancud
Processing Record 22 of Set 1 | tawang
Proces

In [18]:
# Convert array of dictionaries to DataFrame
city_weather_df = pd.DataFrame(city_data)
city_weather_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Portland,US,45.5234,-122.6762,77.63,57,75,8.05,broken clouds
1,Busselton,AU,-33.6500,115.3333,59.58,80,67,20.83,broken clouds
2,Havre-Saint-Pierre,CA,50.2334,-63.5986,66.56,88,75,5.75,broken clouds
3,Ayagoz,KZ,47.9714,80.4392,65.35,36,100,11.52,overcast clouds
4,Mataura,NZ,-46.1927,168.8643,33.48,96,15,2.91,few clouds
...,...,...,...,...,...,...,...,...,...
664,Gejiu,CN,23.3060,103.1635,55.99,94,64,3.09,broken clouds
665,Zaozerne,UA,45.1582,33.2787,73.33,68,16,13.65,few clouds
666,Gasa,PS,31.5000,34.4667,72.88,70,1,4.34,clear sky
667,Novska,HR,45.3406,16.9769,72.18,44,2,1.77,clear sky


In [19]:
# Create the output file (CSV).
output_data_file = "./WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_weather_df.to_csv(output_data_file, index_label="City_ID")